In [1]:
import matplotlib.pyplot as plt
import mplfinance
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import datetime
import yfinance as yf
import time
import stockstats
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from collections import OrderedDict
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#grouping by frequency 
def FrameChanging(df, x):
    t = df.groupby(pd.Grouper(freq=x)).agg({"open": "first", 
                                             "close": "last", 
                                             "low": "min", 
                                             "high": "max",
                                             "volume": "sum"})
    return t

In [5]:
def GetChartPattern(df):
    
    df1 = df.copy()
    
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body

    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as true if body average is > the body length and false if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False


    #setting C_LongBody as true if body average is < the body length and false if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

    #setting C_HasUpShadow as true or false indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)

    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                       
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #setting the further columns as true or false according to the respective conditions    
    df1['C_GravestoneDojiOne'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , True, False)
    df1['C_DragonflyDoji'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_UpShadow'] <= df1['C_Body'] )  )   , True, False)
    
    C_SpinningTopBlackPercent = 34.0
    C_SpinningTopWhitePercent = 34.0
    
    
    
    
    df1['C_IsSpinningTop'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
    
    df1['C_3Dojis'] = np.where( (df1['C_Doji'].shift(2) & df1['C_Doji'].shift(1) & df1['C_Doji'] )  , True, False )
    df1['C_BodyGapUp'] = np.where( (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'] )  , True, False )
    df1['C_BodyGapDn'] = np.where( (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'] )  , True, False )
    
    df1['C_IsSpinningTopWhite'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
   
    
    
    
    df1['C_AbandonedBabyBearish'] = np.where( ( ( df1['C_UpTrend'].shift(2)  & df1['C_WhiteBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['high'].shift(2) <df1['low'].shift(1)) &
                                             df1['C_BlackBody'] & (df1['low'].shift(1) > df1['high'] )   ) )    , "1", "0" )
    
    df1['C_AbandonedBabyBullish'] = np.where( ( ( df1['C_DownTrend'].shift(2)  & df1['C_BlackBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['low'].shift(2) > df1['high'].shift(1)) &
                                             df1['C_WhiteBody'] & (df1['high'].shift(1) < df1['low'] )   ) )    , "1", "0" )
    
    
    
    df1['C_HangingManBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_SmallBody'] & 
                                             df1['C_Body'] > 0 &  (df1['hl2'] < df1['C_BodyLo']) &
                                             (df1['C_DnShadow'] >= (C_Factor * df1['C_Body'])) & (~ df1['C_HasUpShadow'])  ) ), "1", "0" )
    
    
    
    
    df1['C_DojiStarBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'] >  df1['C_BodyHi'].shift(1) )  ) ), "1", "0" )
    
    
    
    df1['C_DojiStarBullish'] = np.where( (( df1['C_DownTrend']  & df1['C_BlackBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'].shift(1) >  df1['C_BodyHi'])  ) ), "1", "0" )
    
    
    
    df1['C_DarkCloudCoverBearish'] = np.where( (  ( df1['C_UpTrend'].shift(1)  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1))     &  ( df1['C_BlackBody'] &
                                             ( df1['open'] >=  df1['high'].shift(1)) & (df1['close']< df1['C_BodyMiddle'].shift(1)) & 
                                                        (df1['close'] > df1['open'].shift(1)) ) ), "1", "0" )
    
    

    
    
    df1['C_DojiRes'] = np.where( ( df1['C_Doji'] & (~ df1['C_DragonflyDoji']) & (~ df1['C_GravestoneDojiOne']) ), "1", "0" )
    


    
    df1['C_EngulfingBearish'] = np.where( ( df1['C_UpTrend']  &  df1['C_BlackBody'] &
                                          df1['C_LongBody'] &  df1['C_WhiteBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] <= df1['open'].shift(1))  &
                                           (df1['open'] >= df1['close'].shift(1)) &
                                          ( (df1['close'] <= df1['open'].shift(1)) | (df1['open'] > df1['close'].shift(1)) ))   , "1", "0")
    
    
    df1['C_EngulfingBullish'] = np.where( ( df1['C_DownTrend']  &  df1['C_WhiteBody'] &
                                           df1['C_LongBody'] &  df1['C_BlackBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] >= df1['open'].shift(1))  &
                                           (df1['open'] <= df1['close'].shift(1)) &
                                          ( (df1['close'] > df1['open'].shift(1)) | (df1['open'] <df1['close'].shift(1)) ))   , "1", "0")
    
    
    
    df1['C_EveningDojiStarBearish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_UpTrend'] &  df1['C_WhiteBody'] &
                                             (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                              df1['C_BlackBody'] &  (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) & 
                                               (df1['C_BodyLo'].shift(2) > df1['C_BodyHi'])), "1", "0" )
    
   


    df1['C_EveningStarBearish'] = np.where((df1['C_LongBody'].shift(2) &
                                            df1['C_SmallBody'].shift(1) & 
                                            df1['C_LongBody'] &
                                           df1['C_UpTrend'] & df1['C_WhiteBody'].shift(2) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                           df1['C_BlackBody'] & 
                                            (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                           (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'])), "1", "0" )
    

    df1['C_FallingWindowBearish'] = np.where((df1['C_DownTrend'].shift(1) &
                                               ( (df1['C_Range'] != 0) & (df1['C_Range'].shift(1) != 0) ) &
                                               (df1['high'] < df1['low'].shift(1)) ), "1", "0" )
    
  
    
    
    df1['C_GravestoneDojiBearish'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , "1", "0")
  
    
    
    
    df1['C_HammerBullish'] = np.where( (  df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                       (df1['C_BodyLo'] > df1['hl2'] ) &
                                      (df1['C_DnShadow'] >= C_Factor * df1['C_Body']) &
                                       (~ df1['C_HasUpShadow']) &
                                       df1['C_DownTrend'] )   , "1", "0")
    
    df1['C_HaramiBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_BlackBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    
    
    
    df1['C_HaramiBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_WhiteBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    
    
    df1['C_HaramiCrossBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_IsDojiBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  

    
    df1['C_HaramiCrossBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_IsDojiBody']  &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  


    df1['C_InvertedHammerBullish'] = np.where( ( df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                              (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                               (~ df1['C_HasDnShadow']))   , "1", "0")
  
    
    
    C_LongLowerShadowPercent = 75.0

    
    df1['C_LongLowerShadowBullish'] = np.where( ( df1['C_DnShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    
    C_LongLowerShadowPercent = 75.0

    df1['C_LongUpperShadowBearish'] = np.where( ( df1['C_UpShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    
    
    C_MarubozuShadowPercentBearish = 5.0

    df1['C_MarubozuBlackBearish'] = np.where( ( df1['C_BlackBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body'])) &
                                            df1['C_BlackBody']  )   , "1", "0")
  
    
    
    C_MarubozuShadowPercentWhite = 5.0
    df1['MarubozuWhiteBullish'] = np.where( ( df1['C_WhiteBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body'])) &
                                            df1['C_WhiteBody']  )   , "1", "0")
  
   
    df1['C_MorningDojiStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    
    
    df1['C_MorningStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_SmallBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    
    
    df1['C_OnNeckBearish'] = np.where((df1['C_DownTrend'] & df1['C_BlackBody'].shift(1) &
                                     df1['C_LongBody'].shift(1) & df1['C_WhiteBody'] & 
                                      (df1['open'] < df1['close'].shift(1)) &
                                     df1['C_SmallBody'] & ( df1['C_Range'] != 0 ) &
                                     (abs(df1['close'] - df1['low'].shift(1)) <= df1['C_BodyAvg']*0.05) ), "1", "0" )
    
    
    df1['C_PiercingBullish'] = np.where( ( (df1['C_DownTrend'].shift(1) & df1['C_BlackBody'].shift(1) & df1['C_LongBody'].shift(1)) &
                                        ( (df1['C_WhiteBody'] & df1['open'] <= df1['low'].shift(1)) & 
                                         (df1['close'] > df1['C_BodyMiddle'].shift(1)) & 
                                         (df1['close'] < df1['open'].shift(1)) )  )  , "1", "0" )
    
    
    df1['C_RisingWindowBullish'] = np.where( ( ( (df1['C_Range'] !=0) & (df1['C_Range'].shift(1)!=0 ) ) &
                                             df1['C_UpTrend'].shift(1) &
                                            df1['low'] > df1['high'].shift(1) )  , "1", "0" )
    
    
    
    df1['C_ShootingStarBearish'] = np.where( (df1['C_SmallBody'] & (df1['C_Body'] > 0 ) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                             (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                            (~ df1['C_HasDnShadow']) )  , "1", "0" )
    
    

   
    df1['C_SpinningTopBlack'] = np.where((df1['C_IsSpinningTop'] & df1['C_BlackBody']) , "1", "0")
    
    
    
    
 
    
   
    df1['C_SpinningTopWhite'] = np.where((df1['C_IsSpinningTopWhite'] & df1['C_WhiteBody']) , "1", "0")
    
    
    
    
    
   

    df1['C_TriStarBearish'] = np.where(( df1['C_3Dojis'] &  df1['C_UpTrend'].shift(2) &
                                       df1['C_BodyGapUp'].shift(1) & df1['C_BodyGapDn'] ) , "1", "0")
    
    
    df1['C_TriStarBullish'] = np.where(( df1['C_3Dojis'] &  df1['C_DownTrend'].shift(2) &
                                       df1['C_BodyGapDn'].shift(1) & df1['C_BodyGapUp'] ) , "1", "0")
    
    
    
    df1['C_TweezerBottomBullish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['low']-df1['low'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_BlackBody'].shift(1) & df1['C_WhiteBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    df1['C_TweezerTopBearish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['high']-df1['high'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_WhiteBody'].shift(1) & df1['C_BlackBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    
    
    return df1

In [6]:
#for all the mentioned columns the values are normallised 
def normalize(df):
    result = df.copy()
    for feature_name in ['open','close','low','high','volume','MA100','MA200','MA50','Return', 'C_BodyHi', 'C_BodyLo', 'hl2']:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [7]:
def Preprocess(df):
    
    df['DateIndex'] = pd.to_datetime(df['timestamp'])#convert timestamp to date time formant and save it to date index
    df = df.set_index('DateIndex')#set dateindex as index of data frame
    df = df.drop(['timestamp'], axis=1)
    df1 = FrameChanging(df, '30Min')#grouping by frequeny of 30 minutes
    df1 = df1.dropna()
    #calculating lag,return,P_result1,P_result0 
    df1['L1'] = df1['close'].shift(1)
    df1['Return'] = (df1['close']-df1['L1'])
    df1 = df1.dropna()
    
    df1.loc[df1['Return']>=0, 'Result_1'] = '1'
    df1.loc[df1['Return']<0, 'Result_1'] = '0'
    
    df1.loc[df1['Return']<0, 'Result_0'] = '1'
    df1.loc[df1['Return']>=0, 'Result_0'] = '0'
    
    
    
    df1['P_Result_1'] = df1['Result_1'].shift(-1)
    df1['P_Result_0'] = df1['Result_0'].shift(-1)
    
    df1 = df1.drop(['Result_1', 'Result_0'], axis=1)
    
    dfR = GetChartPattern(df1)#generating all the candlestick pattern columns in the function
    
    dfRS = normalize(dfR)#normalise the data frame
    #replace true with 1 and false with 0
    dfRS = dfRS.replace(True,1)
    dfRS = dfRS.replace(False,0)
    
    return dfRS
    
    

In [8]:
import glob
l =(glob.glob("../Data/*.csv"))
print(l[1])

../Data/TORNTPHARM__EQ__NSE__NSE__MINUTE.csv


In [9]:
# df = pd.read_csv("../Data/ADANIPOWER__EQ__NSE__NSE__MINUTE.csv")

In [10]:
# "../Data/RELIANCE__EQ__NSE__NSE__MINUTE.csv"

In [11]:
df = pd.read_csv("../Data/RELIANCE__EQ__NSE__NSE__MINUTE.csv")

In [12]:
df1 = Preprocess(df)

In [13]:
df1.head()

,open,close,low,high,volume,L1,Return,P_Result_1,P_Result_0,MA100,MA200,MA50,C_DownTrend,C_UpTrend,C_BodyHi,C_BodyLo,C_Body,C_BodyAvg,C_SmallBody,C_LongBody,C_UpShadow,C_DnShadow,C_HasUpShadow,C_HasDnShadow,C_WhiteBody,C_BlackBody,C_Range,C_BodyMiddle,C_ShadowEquals,C_IsDojiBody,C_Doji,hl2,C_GravestoneDojiOne,C_DragonflyDoji,C_IsSpinningTop,C_3Dojis,C_BodyGapUp,C_BodyGapDn,C_IsSpinningTopWhite,C_AbandonedBabyBearish,C_AbandonedBabyBullish,C_HangingManBearish,C_DojiStarBearish,C_DojiStarBullish,C_DarkCloudCoverBearish,C_DojiRes,C_EngulfingBearish,C_EngulfingBullish,C_EveningDojiStarBearish,C_EveningStarBearish,C_FallingWindowBearish,C_GravestoneDojiBearish,C_HammerBullish,C_HaramiBearish,C_HaramiBullish,C_HaramiCrossBearish,C_HaramiCrossBullish,C_InvertedHammerBullish,C_LongLowerShadowBullish,C_LongUpperShadowBearish,C_MarubozuBlackBearish,MarubozuWhiteBullish,C_MorningDojiStarBullish,C_MorningStarBullish,C_OnNeckBearish,C_PiercingBullish,C_RisingWindowBullish,C_ShootingStarBearish,C_SpinningTopBlack,C_SpinningTopWhite,C_TriStarBearish,C_TriStarBullish,C_TweezerBottomBullish,C_TweezerTopBearish
DateIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-02 09:30:00+05:30,0.023805,0.026863,0.024957,0.027306,0.018131,538.68,0.541044,1,0,0.023685,0.023507,0.025428,0,0,0.026603,0.024004,3.92,3.920000,0,0,1.75,0.18,1,1,1,0,5.85,0.007250,1,0,0,0.025919,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2017-01-02 10:00:00+05:30,0.026844,0.027716,0.027891,0.026842,0.006255,542.60,0.527538,0,1,0.024145,0.023996,0.025874,0,0,0.027456,0.027069,1.10,2.409286,1,0,0.05,0.35,1,1,1,0,1.50,0.002020,1,0,0,0.027144,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2017-01-02 10:30:00+05:30,0.027619,0.027584,0.027891,0.026804,0.005304,543.70,0.521456,1,0,0.024251,0.024108,0.025977,0,0,0.027378,0.027796,0.07,1.515671,1,0,0.10,1.28,0,1,0,1,1.45,0.000128,1,1,1,0.027124,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-01-02 11:00:00+05:30,0.027720,0.027910,0.028790,0.027422,0.008380,543.53,0.524282,1,0,0.024392,0.024258,0.026114,0,0,0.027650,0.027952,0.22,1.119288,1,0,0.55,0.33,1,1,1,0,1.10,0.000403,1,0,0,0.027883,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2017-01-02 11:30:00+05:30,0.028085,0.028477,0.029221,0.027808,0.019742,543.95,0.525766,0,1,0.024598,0.024477,0.026314,0,0,0.028216,0.028320,0.48,0.952499,1,0,0.32,0.25,1,1,1,0,1.05,0.000879,1,0,0,0.028291,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [14]:
list(df1)

['open',
 'close',
 'low',
 'high',
 'volume',
 'L1',
 'Return',
 'P_Result_1',
 'P_Result_0',
 'MA100',
 'MA200',
 'MA50',
 'C_DownTrend',
 'C_UpTrend',
 'C_BodyHi',
 'C_BodyLo',
 'C_Body',
 'C_BodyAvg',
 'C_SmallBody',
 'C_LongBody',
 'C_UpShadow',
 'C_DnShadow',
 'C_HasUpShadow',
 'C_HasDnShadow',
 'C_WhiteBody',
 'C_BlackBody',
 'C_Range',
 'C_BodyMiddle',
 'C_ShadowEquals',
 'C_IsDojiBody',
 'C_Doji',
 'hl2',
 'C_GravestoneDojiOne',
 'C_DragonflyDoji',
 'C_IsSpinningTop',
 'C_3Dojis',
 'C_BodyGapUp',
 'C_BodyGapDn',
 'C_IsSpinningTopWhite',
 'C_AbandonedBabyBearish',
 'C_AbandonedBabyBullish',
 'C_HangingManBearish',
 'C_DojiStarBearish',
 'C_DojiStarBullish',
 'C_DarkCloudCoverBearish',
 'C_DojiRes',
 'C_EngulfingBearish',
 'C_EngulfingBullish',
 'C_EveningDojiStarBearish',
 'C_EveningStarBearish',
 'C_FallingWindowBearish',
 'C_GravestoneDojiBearish',
 'C_HammerBullish',
 'C_HaramiBearish',
 'C_HaramiBullish',
 'C_HaramiCrossBearish',
 'C_HaramiCrossBullish',
 'C_InvertedHammerB

In [15]:
column_list = ['C_GravestoneDojiOne',
 'C_DragonflyDoji',
 'C_IsSpinningTop',
 'C_3Dojis',
 'C_BodyGapUp',
 'C_BodyGapDn',
 'C_IsSpinningTopWhite',
 'C_AbandonedBabyBearish',
 'C_AbandonedBabyBullish',
 'C_HangingManBearish',
 'C_DojiStarBearish',
 'C_DojiStarBullish',
 'C_DarkCloudCoverBearish',
 'C_DojiRes',
 'C_EngulfingBearish',
 'C_EngulfingBullish',
 'C_EveningDojiStarBearish',
 'C_EveningStarBearish',
 'C_FallingWindowBearish',
 'C_GravestoneDojiBearish',
 'C_HammerBullish',
 'C_HaramiBearish',
 'C_HaramiBullish',
 'C_HaramiCrossBearish',
 'C_HaramiCrossBullish',
 'C_InvertedHammerBullish',
 'C_LongLowerShadowBullish',
 'C_LongUpperShadowBearish',
 'C_MarubozuBlackBearish',
 'MarubozuWhiteBullish',
 'C_MorningDojiStarBullish',
 'C_MorningStarBullish',
 'C_OnNeckBearish',
 'C_PiercingBullish',
 'C_RisingWindowBullish',
 'C_ShootingStarBearish',
 'C_SpinningTopBlack',
 'C_SpinningTopWhite',
 'C_TriStarBearish',
 'C_TriStarBullish',
 'C_TweezerBottomBullish',
 'C_TweezerTopBearish']

In [16]:
col = ['open',
 'close',
 'low',
 'high',
 'volume',
 'L1',
 'Return',
 'MA100',
 'MA200',
 'MA50',
 'C_DownTrend',
 'C_UpTrend',
 'C_BodyHi',
 'C_BodyLo',
 'C_Body',
 'C_BodyAvg',
 'C_SmallBody',
 'C_LongBody',
 'C_UpShadow',
 'C_DnShadow',
 'C_HasUpShadow',
 'C_HasDnShadow',
 'C_WhiteBody',
 'C_BlackBody',
 'C_Range',
 'C_BodyMiddle',
 'C_ShadowEquals',
 'C_IsDojiBody',
 'C_Doji',
 'hl2',
 'C_GravestoneDojiOne',
 'C_DragonflyDoji',
 'C_IsSpinningTop',
 'C_3Dojis',
 'C_BodyGapUp',
 'C_BodyGapDn',
 'C_IsSpinningTopWhite',
 'C_AbandonedBabyBearish',
 'C_AbandonedBabyBullish',
 'C_HangingManBearish',
 'C_DojiStarBearish',
 'C_DojiStarBullish',
 'C_DarkCloudCoverBearish',
 'C_DojiRes',
 'C_EngulfingBearish',
 'C_EngulfingBullish',
 'C_EveningDojiStarBearish',
 'C_EveningStarBearish',
 'C_FallingWindowBearish',
 'C_GravestoneDojiBearish',
 'C_HammerBullish',
 'C_HaramiBearish',
 'C_HaramiBullish',
 'C_HaramiCrossBearish',
 'C_HaramiCrossBullish',
 'C_InvertedHammerBullish',
 'C_LongLowerShadowBullish',
 'C_LongUpperShadowBearish',
 'C_MarubozuBlackBearish',
 'MarubozuWhiteBullish',
 'C_MorningDojiStarBullish',
 'C_MorningStarBullish',
 'C_OnNeckBearish',
 'C_PiercingBullish',
 'C_RisingWindowBullish',
 'C_ShootingStarBearish',
 'C_SpinningTopBlack',
 'C_SpinningTopWhite',
 'C_TriStarBearish',
 'C_TriStarBullish',
 'C_TweezerBottomBullish',
 'C_TweezerTopBearish']

In [17]:
len(col)

72

In [34]:
#preprocessing data and defining dependent and indeoendent variables for training and testing
def PrepareData(df):
    
    df1 = df.copy()
    df1[column_list] = df1[column_list].astype(int)
    df1["sum"] = df1[column_list].sum(axis=1)
    
    Y = df1[['P_Result_0', 'P_Result_1']].to_numpy()
    Z = df1["sum"].to_numpy()
    df1 = df1.drop(['P_Result_1','P_Result_0','sum'], axis=1)
    X = df1[col].to_numpy()
    
    seq_len = 6
    
    x = []
    y = []

    for i in range(seq_len, len(X)):
        
        if(Z[i]>0):
            x.append(X[i-seq_len:i])
            y.append(Y[i])
    print("Y",Y)
    return x,y

In [35]:
# df1[column_list]


In [36]:
Xx, Yy = PrepareData(df1)


Y [['0' '1']
 ['1' '0']
 ['0' '1']
 ...
 ['1' '0']
 ['0' '1']
 [nan nan]]


In [37]:
# print(Xx,Yy)

In [20]:
len(Xx[0][0])

72

In [21]:
len(Yy)

11133

In [22]:
x1 = np.asarray(Xx).astype('float32')
y1 = np.asarray(Yy).astype('float32')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x1, y1, train_size=0.8, random_state=42)
n_timesteps, n_features, n_outputs = 6, len(col), 2


In [23]:
from tensorflow import keras
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
import keras.models
from keras.layers import Activation, Dense
from keras.models import Sequential

In [24]:
epochs = 20
batch_size =64


In [25]:
#training LSTM model 
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-09-02 18:52:49.991804: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
# fit network
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
          verbose=0, validation_data=(X_test, y_test))

In [27]:
history.history

{'loss': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'accuracy': [0.5097687244415283,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677,
  0.5103301405906677],
 'val_loss': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 'val_accuracy': [0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.5042658448219299,
  0.50426

In [28]:
# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
Y_pred = model.predict(X_test)

In [29]:
accuracy

0.5042658448219299